In [1]:
import calendar
import json
import os
import time
import traceback
from datetime import datetime

import pandas as pd
import requests
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

spark = (
    SparkSession.builder
        .master("local")
        .appName("TFL Notebook")
        .getOrCreate()
)

trips = spark.read.parquet("data/parquet_trip")

trips.limit(10).toPandas()


,rental_id,duration,bike_id,end_station_id,end_station_name,start_station_id,start_station_name,start_ts,start_year,start_month,start_day,start_day_of_week,start_hour,end_ts,end_year,end_month,end_day,end_day_of_week,end_hour
0,64401354,1080,7983,325,"St. Martin's Street, West End",173,"Waterloo Road, South Bank",2017-04-26 00:00:00,2017,4,26,4,0,2017-04-26 00:18:00,2017,4,26,4,0
1,64401355,180,11396,328,"New North Road 2, Hoxton",539,"Hoxton Station, Hoxton",2017-04-26 00:00:00,2017,4,26,4,0,2017-04-26 00:03:00,2017,4,26,4,0
2,64401356,240,10399,219,"Bramham Gardens, Earl's Court",187,"Queen's Gate (South), South Kensington",2017-04-26 00:00:00,2017,4,26,4,0,2017-04-26 00:04:00,2017,4,26,4,0
3,64401357,660,3725,76,"Longford Street, The Regent's Park",804,"Good's Way, King's Cross",2017-04-26 00:00:00,2017,4,26,4,0,2017-04-26 00:11:00,2017,4,26,4,0
4,64401358,1080,1476,325,"St. Martin's Street, West End",173,"Waterloo Road, South Bank",2017-04-26 00:01:00,2017,4,26,4,0,2017-04-26 00:19:00,2017,4,26,4,0
5,64401359,1080,14568,325,"St. Martin's Street, West End",173,"Waterloo Road, South Bank",2017-04-26 00:01:00,2017,4,26,4,0,2017-04-26 00:19:00,2017,4,26,4,0
6,64401360,480,487,737,"Fulham Broadway, Walham Green",430,"South Parade, Chelsea",2017-04-26 00:01:00,2017,4,26,4,0,2017-04-26 00:09:00,2017,4,26,4,0
7,64401361,420,5398,320,"Queen Mother Sports Centre, Victoria",74,"Vauxhall Cross, Vauxhall",2017-04-26 00:01:00,2017,4,26,4,0,2017-04-26 00:08:00,2017,4,26,4,0
8,64401362,1020,5710,325,"St. Martin's Street, West End",173,"Waterloo Road, South Bank",2017-04-26 00:01:00,2017,4,26,4,0,2017-04-26 00:18:00,2017,4,26,4,0
9,64401363,540,13051,444,"Bethnal Green Garden, Bethnal Green",588,"Hoxton Street, Hoxton",2017-04-26 00:01:00,2017,4,26,4,0,2017-04-26 00:10:00,2017,4,26,4,0


In [2]:
(
    trips
    .groupby('start_year', 'start_month')
    .agg({
        "*": "count",
        "duration": "avg",
    })
    .toPandas()
    .sort_values(['start_year', 'start_month'], ascending=[True, True])
)

,start_year,start_month,avg(duration),count(1)
24,2015,1,1148.544935,546113
0,2015,2,1162.760531,542526
18,2015,3,1214.140998,694364
6,2015,4,1550.291654,829921
25,2015,5,1735.941579,892692
20,2015,6,1536.338353,1030640
19,2015,7,1722.151621,1122670
7,2015,8,1777.191664,1030760
11,2015,9,1391.295199,894635
12,2015,10,1349.525099,876245
